test cases:

1. get the expected output
2. can throw out errors (unfitted, format error, etc.)
3. produce correct (or nearly correct) amount of NAs

In [1]:
import pandas as pd
import numpy as np

In [2]:
%cd /Users/alex/PETsARD

/Users/alex/PETsARD


/Users/alex/miniforge3/envs/dev/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
from PETsARD import (
    Loader,
    Processor,
    Synthesizer,
    Evaluator,
    Executor
)

In [7]:
load = Loader(
    filepath='benchmark://adult-income'
)
load.load()

Loader - Benchmarker: file benchmark/adult-income.csv already exist and match SHA-256.
                      PETsARD will ignore download and use local data directly.


In [4]:
synth = Synthesizer(method='sdv-gaussiancopula')

In [5]:
synth.create(load.data, metadata=load.metadata.to_sdv())

Synthesizer (SDV - SingleTable): Metafile loading time: 0.0001 sec.


In [6]:
synth.fit_sample()

Synthesizer (SDV - SingleTable): Fitting GaussianCopula.
Synthesizer (SDV - SingleTable): Fitting  GaussianCopula spent 6.6271 sec.


/Users/alex/miniforge3/envs/dev/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:700: RuntimeWarning: invalid value encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)


Synthesizer (SDV - SingleTable): Sampling GaussianCopula # 48842 rows (same as raw) in 0.864 sec.


In [9]:
import tempfile

In [22]:
yaml_text = """
---
Loader:
    adult-income:
        filepath: 'test.csv'
Splitter:
    p80:
        num_samples: 1
        train_split_ratio: 0.8
Preprocessor:
    default-smartnoise:
        sequence:
            - 'missing'
            - 'outlier'
            - 'encoder'
            - 'scaler'
Synthesizer:
    smartnoise-dpctgan03:
        method: 'smartnoise-dpctgan'
        epsilon: 0.3
        epochs: 1000
        batch_size: 10
Postprocessor:
    default-smartnoise:
        method: 'default'
Evaluator:
    anonymeter-singlingout:
        method: 'anonymeter-singlingout'
        n_attacks: 10
Reporter:
    save_report_global:
        method: 'save_report'
        output: '20240328_ano'
        granularity: 'global'
...
"""


with tempfile.NamedTemporaryFile(mode='w+', delete=True) as tmpfile:
    tmpfile.write(yaml_text)
    tmpfile.flush()
    tmpfile.seek(0)

    exec = Executor(config=tmpfile.name)
    exec.run()

Now is Loader with adult-income...
Now is Splitter with p80_[1-1]...
Now is Preprocessor with default-smartnoise...
Now is Synthesizer with smartnoise-dpctgan03...
Synthesizer (SmartNoise): Fitting dpctgan.


/Users/alex/miniforge3/envs/dev/lib/python3.10/site-packages/opacus/privacy_engine.py:638: UserWarning: The sample rate will be defined from ``batch_size`` and ``sample_size``.The returned privacy budget will be incorrect.
  warnings.warn(
/Users/alex/miniforge3/envs/dev/lib/python3.10/site-packages/opacus/privacy_engine.py:229: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
  warnings.warn(
/Users/alex/miniforge3/envs/dev/lib/python3.10/site-packages/torch/nn/modules/module.py:1359: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward

Epoch 1, Loss G: 0.6573, Loss D: 1.3684
epsilon is 0.023191678421611878, alpha is 63.0
Epoch 2, Loss G: 0.6580, Loss D: 1.3673
epsilon is 0.24137717571820097, alpha is 24.0
Synthesizer (SmartNoise): Fitting  dpctgan spent 0.485 sec.
Synthesizer (SmartNoise): Sampling dpctgan # 80 rows (same as raw) in 0.0095 sec.
Now is Postprocessor with default-smartnoise...
Now is Evaluator with anonymeter-singlingout...
Now is Reporter with save_report_global...
Now is 20240328_ano[Report]_[global] save to csv...


In [12]:
df = pd.DataFrame({'A': np.random.rand(100), 'B': np.random.rand(100), 'C': np.random.rand(100)})

In [14]:
df.to_csv('test.csv', index=False)

In [17]:
exec.status.status

{'Loader': {'expt': 'adult-income',
  'operator': <PETsARD.operator.LoaderOperator at 0x17ed28b50>},
 'Splitter': {'expt': 'p80_[1-1]',
  'operator': <PETsARD.operator.SplitterOperator at 0x17ed28970>}}